In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.7:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1642747679860)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@eed54e5


이 장에서는 반드시 이해해야 할 스파크 SQL의 핵심 개념을 설명함<br/>

스파크 SQL을 사용해 데이터베이스에 생성된 뷰나 테이블에 SQL 쿼리를 실행할 수 있음<br/>
또한 시스템 함수를 사용하거나 사용자 정의 함수를 정의할 수 있음<br/>
그리고 워크로드를 최적화하기 위해 쿼리 실행 계획을 분석할 수도 있음<br/>
**스파크 SQL은 DataFrame과 Dataset API에 통합되어 있음<br/>
따라서 데이터 변환 시 SQL과 DataFrame의 기능을 모두 사용할 수 있으며 두 방식 모두 동일한 실행 코드로 컴파일됨**<br/>

# 10.1 SQL이란

SQL 또는 구조적 질의 언어(Structured Query Language)는 데이터에 대한 관계형 연산을 표현하기 위한 도메인 특화 언어임<br/>
모든 관계형 데이터베이스에서 사용되며, 많은 'NoSQL' 데이터베이스에서도 쉽게 사용할 수 있는 변형된 자체 SQL을 제공함<br/>
기술 전문가들은 SQL이 사라질 것이라고 예상했지만 여전히 많은 기업에서 사용하고 있는 매우 우수한 데이터 처리 언어임<br/>
스파크는 ANSI SQL:2003(대부분의 SQL 데이터베이스에서 채택하고 있는 표준)의 일부를 구현했음<br/>
그러므로 스파크는 유명한 벤치마크인 TPC-DS를 성공적으로 통과할 수 있음<br/>

# 10.2 빅데이터와 SQL: 아파치 하이브

스파크가 등장하기 전에는 하이브가 빅데이터 SQL 접근 계층에서 사실상의 표준이었음<br/>
*페이스북에서 최초로 개발한 하이브는 SQL 처리가 필요한 빅데이터 업계에서 인기 있는 도구가 되었음<br/>
분석가들이 하이브로 SQL 쿼리를 실행할 수 있게 되면서 하둡을 다양한 산업군으로 진출시키는 데 다방면으로 도움을 줬음*<br/>
스파크는 RDD를 이용하는 범용 처리 엔진으로 시작했지만 이제 많은 사용자가 스파크 SQL을 사용하고 있음<br/>

# 10.3 빅데이터와 SQL: 스파크 SQL

스파크 2.0 버전에는 하이브를 지원할 수 있는 상위 호환 기능으로 ANSI-SQL과 HiveQL을 모두 지원하는 자체 개발된 SQL 파서가 포함되어 있음<br/>
스파크 SQL은 DataFrame과의 뛰어난 호환성 덕분에 다양한 기업에서 강력한 기능으로 자리매김할 것<br/>
ex) 2016년 말 페이스북은 스파크 워크로드를 가동하기 시작했으며 스파크 워크로드 덕분에 큰 효과를 봤다고 발표했음<br/>
다음은 블로그 내용<br/>

*우리는 수백 개의 하이브 잡으로 구성된 파이프라인을 단일 스파크 잡으로 대체하기 위해 스파크에 도전했다.<br/>
그리고 우리는 다양한 성능 및 신뢰도 개선 작업을 거쳐 운영 환경의 개체 데이터 순위 처리 시스템 중 하나를 스파크로 처리할 수 있게 되었다.<br/>
스파크 기반 파이프라인은 기존의 하이브 기반 파이프라인에 비해 엄청난 성능 개선(4.5~6배의 CPU 성능 개선, 3~4배의 자원 예약 개선, 최대 5배의 지연 시간 감소)를 만들어냈으며 수개월째 운영 환경에서 잘 동작하고 있다.*

스파크 SQL의 강력함은 몇 가지 핵심 요인에서 비롯됨<br/>
* SQL 분석가들은 쓰리프트 서버나 SQL 인터페이스에 접속해 스파크의 연산 능력을 활용할 수 있음
* 데이터 엔지니어와 데이터 사이언티스트는 전체 데이터 처리 파이프라인에 스파크 SQL을 사용할 수 있음

이 통합형 API는 SQL로 데이터를 조회하고 DataFrame으로 변환한 다음 스파크의 MLlib이 제공하는 대규모 머신러닝 알고리즘 중 하나를 사용해 수행한 결과를 다른 데이터 소스에 저장하는 전체 과정을 가능하게 만듦<br/>

*NOTE*<br/>
스파크 SQL은 온라인 트랜잭션 처리(online transaction processing, OLTP) 데이터베이스가 아닌 온라인 분석용(online analytic processing, OLAP) 데이터베이스로 동작함<br/>
즉, 매우 낮은 지연 시간이 필요한 쿼리를 수행하기 위한 용도로 사용할 수 없음<br/>
언젠가는 '인플레이스 수정(in-place modification)' 방식을 지원하겠지만 현재는 사용할 수 없음<br/>

## 10.3.1 스파크와 하이브의 관계

**스파크 SQL은 하이브 메타스토어를 사용하므로 하이브와 잘 연동할 수 있음**<br/>
하이브 메타스토어는 여러 세션에서 사용할 테이블 정보를 보관하고 있음<br/>
스파크 SQL은 하이브 메타스토어에 접속(이미 하이브를 사용하고 있는 경우)한 뒤 조회할 파일 수를 최소화하기 위해 메타데이터를 참조함<br/>
이 기능은 기존 하둡 환경의 모든 워크로드를 스파크로 이관하려는 사용자들에게 인기를 얻고 있음<br/>

### 하이브 메타스토어

하이브 메타스토어에 접속하려면 몇 가지 속성이 필요함<br/>
먼저 접근하려는 하이브 메타스토어에 적합한 버전을 spark.sql.hive.metastore.version에 설정해야 함<br/>
기본값은 1.2.1임<br/>
또한 HiveMetastoreClient가 초기화되는 방식을 변경하려면 spark.sql.hive.metastore.jars를 설정해야 함<br/>
스파크는 기본 버전을 사용하지만 메이븐 저장소나 자바 가상 머신의 표준 포맷에 맞게 클래스패스를 정의할 수도 있음<br/>
또한 하이브 메타스토어가 저장된 다른 데이터베이스에 접속하려면 적합한 클래스 접두사(MySQL 사용하려면 com.mysql.jdbc로 명시함)를 정의해야 함<br/>
스파크와 하이브에서 공유할 수 있도록 클래스 접두사를 spark.sql.hive.metastore.sharedPrefixes 속성에 설정함<br/>

# 10.4 스파크 SQL 쿼리 실행 방법

스파크는 SQL 쿼리를 실행할 수 있는 몇 가지 인터페이스를 제공함<br/>

## 10.4.1 스파크 SQL CLI

스파크 SQL CLI는 로컬 환경의 명령행에서 기본 스파크 SQL 쿼리를 실행할 수 있는 편리한 도구임<br/>
스파크 SQL CLI는 쓰리프트 JDBC 서버와 통신할 수 없음<br/>
스파크 SQL CLI를 사용하려면 스파크 이렉터리에서 다음 명령을 실행함<br/>

In [2]:
/*
./bin/spark-sql
*/

스파크가 설치된 경로의 conf 디렉터리에 hive-site.xml, core-site.xml, hdfs-site.xml 파일을 배치해 하이브를 사용할 수 있는 환경을 구성할 수 있음<br/>

## 10.4.2 스파크의 프로그래밍 SQL 인터페이스

서버를 설정해 SQL을 사용할 수도 있지만, 스파크에서 지원하는 언어 API로 비정형 SQL을 실행할 수도 있음<br/>
이를 위해 SparkSession 객체의 sql 메서드를 사용함<br/>
처리된 결과는 DataFrame을 반환함<br/>
예를 들어 파이썬이나 스칼라에서는 다음과 같은 실행할 수 있음<br/>

In [3]:
spark.sql("SELECT 1 + 1").show()

+-------+
|(1 + 1)|
+-------+
|      2|
+-------+



spark.sql("SELECT 1 + 1") 명령은 프로그래밍 방식으로 평가할 수 있는 DataFrame을 반환함<br/>
다른 트랜스포메이션과 마찬가지로 즉시 실행되지 않고 지연 처리됨<br/>
또한 DataFrame을 사용하는 것보다 SQL 코드로 표현하기 훨씬 쉬운 트랜스포메이션이기 때문에 엄청나게 강력한 인터페이스임<br/>

함수에 여러 줄로 구성된 문자열을 전달할 수 있으므로 여러 줄로 구성된 쿼리를 아주 간단히 표현할 수 있음<br/>
예를 들어 파이썬이나 스칼라에서 다음과 같은 코드를 실행할 수 있음<br/>

In [4]:
/*
spark.sql("""SELECT user_id, department, first_name FROM professors
    WHERE department IN
        (SELECT name FROM department WHERE created_date >= '2016-01-01')""")
*/

**SQL과 DataFrame은 완벽하게 연동될 수 있으므로 더 강력함**<br/>
예를 들어 DataFrame을 생성하고 SQL을 사용해 처리할 수 있으며 그 결과를 다시 DataFrame으로 돌려받게 됨<br/>
이러한 방식은 다양한 처리에 자주 사용하게 될 매우 효과적인 패턴 중 하나임<br/>

In [43]:
spark.read.json("Downloads/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json")
    .createOrReplaceTempView("some_sql_view") // DataFrame을 SQL에서 사용할 수 있도록 처리

spark.sql(
"""
SELECT DEST_COUNTRY_NAME, sum(count)
FROM some_sql_view GROUP BY DEST_COUNTRY_NAME
""").show()

+--------------------+----------+
|   DEST_COUNTRY_NAME|sum(count)|
+--------------------+----------+
|            Anguilla|        41|
|              Russia|       176|
|            Paraguay|        60|
|             Senegal|        40|
|              Sweden|       118|
|            Kiribati|        26|
|              Guyana|        64|
|         Philippines|       134|
|            Djibouti|         1|
|            Malaysia|         2|
|           Singapore|         3|
|                Fiji|        24|
|              Turkey|       138|
|                Iraq|         1|
|             Germany|      1468|
|              Jordan|        44|
|               Palau|        30|
|Turks and Caicos ...|       230|
|              France|       935|
|              Greece|        30|
+--------------------+----------+
only showing top 20 rows



## 10.4.3 스파크 SQL 쓰리프트 JDBC/ODBC 서버

스파크는 자바 데이터베이스 연결(Java Database Connectivity, JDBC) 인터페이스를 제공함<br/>
사용자나 원격 프로그램은 스파크 SQL을 실행하기 위해 이 인터페이스로 스파크 드라이버에 접속함<br/>
비즈니스 분석가가 태블로(Tableau) 같은 비즈니스 인텔리전스 소프트웨어를 이용해 스파크에 접속하는 형태가 가장 대표적인 활용 사례<br/>
쓰리프트 JDBC/ODBC(Open Database Connectivity) 서버는 하이브 1.2.1 버전의 HiveServer2에 맞추어 구현되어 있음<br/>
스파크나 하이브 1.2.1 버전에 있는 beeline 스크립트를 이용해 JDBC 서버를 테스트해볼 수 있음<br/>

# 10.5 카탈로그

스파크 SQL에서 가장 높은 추상화 단계는 카탈로그(catalog)임<br/>
*카탈로그는 테이블에 저장된 데이터에 대한 메타데이터뿐만 아니라 데이터베이스, 테이블, 함수 그리고 뷰에 대한 정보를 추상화함*<br/>
카탈로그는 org.apache.spark.sql.catalog.Catalog 패키지로 사용할 수 있음<br/>
카탈로그는 테이블, 데이터베이스 그리고 함수를 조회하는 등 여러 가지 유용한 함수를 제공함<br/>
이와 관련된 내용은 잠시 후에 알아보겠음<br/>

# 10.6 테이블

스파크SQL을 사용해 작업을 수행하려면 먼저 테이블을 정의해야 함<br/>
테이블은 명령을 실행할 데이터의 구조라는 점에서 DataFrame과 논리적으로 동일함<br/>
테이블을 사용해 9장에서 알아본 조인, 필터링, 집계 등 여러 데이터 변환 작업을 수행할 수 있음<br/>
DataFrame은 프로그래밍 언어로 정의하지만 테이블은 데이터베이스에서 정의함<br/>
스파크에서 테이블을 생성하면 **default** 데이터베이스에 등록됨<br/>

스파크 2.x 버전에서 테이블은 **항상 데이터를 가지고 있다**는 점을 반드시 기억해야 함<br/>
임시 테이블의 개념이 없으며 데이터를 가지지 않은 뷰만 존재함<br/>
테이블을 제거하면 모든 데이터가 제거되므로 조심해야 함<br/>

## 10.6.1 스파크 관리형 테이블

**관리형 테이블**과 **외부 테이블**의 개념은 반드시 기억해두어야 함<br/>
테이블은 두 가지 중요한 정보를 저장함<br/>
테이블의 데이터와 테이블에 대한 데이터, 즉 **메타데이터**임<br/>
스파크는 데이터뿐만 아니라 파일에 대한 메타데이터를 관리할 수 있음<br/>
*디스크에 저장된 파일을 이용해 테이블을 정의하면 외부 테이블을 정의하는 것임*<br/>
*DataFrame의 saveAsTable 메서드는 스파크가 관련된 모든 정보를 추적할 수 있는 관리형 테이블을 만들 수 있음*<br/>

saveAsTable 메서드는 테이블을 읽고 데이터를 스파크 포맷으로 변환한 후 새로운 경로에 저장함<br/>
새로운 실행 계획에 이러한 동작이 반영되어 있음을 알 수 있으며, 하이브의 기본 웨어하우스 경로에 데이터를 저장하는 것을 확인할 수 있음<br/>
데이터 저장 경로를 변경하려면 SparkSession을 생성할 때 spark.sql.warehouse.dir 속성에 원하는 디렉터리 경로를 설정함<br/>
기본 저장 경로는 /use/hive/warehouse임<br/>

저장 경로 하위에서 데이터베이스 목록을 확인할 수 있음<br/>
스파크는 데이터베이스 개념도 존재함<br/>
10.8절 '데이터베이스'에서 자세히 알아보겠음<br/>
show tables IN *databaseName* 쿼리를 사용해 특정 데이터베이스의 테이블을 확인할 수도 있음<br/>
쿼리에서 *databaseName* 부분은 테이블을 조회할 데이터베이스 이름을 나타냄<br/>

신규 클러스터 로컬 모드에서 실행하면 빈 테이블 목록을 반환함<br/>

## 10.6.2 테이블 생성하기

다양한 데이터 소스를 사용해 테이블을 생성할 수 있음<br/>
스파크는 SQL에서 전체 데이터 소스 API를 재사용할 수 있는 독특한 기능을 지원함<br/>
즉, 테이블을 정의한 다음 테이블에 데이터를 적재할 필요가 없음<br/>
스파크는 실행 즉시 테이블을 생성함<br/>
파일에서 데이터를 읽을 때 모든 종류의 정교한 옵션을 지정할 수도 있음<br/>
예를 들어 9장에서 사용한 항공운항 데이터를 읽는 방법은 다음과 같음<br/>

In [128]:
spark.sql(
"""
CREATE TABLE IF NOT EXISTS flights(
    DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)
USING JSON OPTIONS (path './Downloads/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json')
""").show()

++
||
++
++



**USING**과 **STORED AS** 구문<br/>
위 예제에서 USING 구문은 매우 중요함<br/>
포맷을 지정하지 않으면 스파크는 기본적으로 하이브 SerDe 설정을 사용함<br/>
하이브 SerDe는 스파크의 자체 직렬화보다 훨씬 느리므로 테이블을 사용하는 Reader와 Write 성능에 영향을 미침<br/>
하이브 사용자는 STORED AS 구문으로 하이브 테이블을 생성할 수 있음<br/>

In [129]:
// 특정 테이블에서 원하는 데이터만 추출해서 새로운 테이블을 생성(CTAS 패턴)
spark.sql( 
"""
CREATE TABLE IF NOT EXISTS flights_from_select
USING Parquet AS SELECT * FROM flights
""").show()
// spark-warehouse 폴더에 테이블 이름의 서브폴더가 생성되고 그 안에 파케이 파일이 저장됨

++
||
++
++



임시 테이블 개념은 현재 스파크에 존재하지 않음<br/>
사용자는 임시 뷰를 만들어 이 기능을 사용할 수 있음<br/>
자세한 내용은 10.7절 '뷰'를 참조할 것<br/>

## 10.6.3 외부 테이블 생성하기

하이브는 초기 빅데이터 SQL 시스템 중 하나였음<br/>
그리고 스파크 SQL은 완벽하게 하이브 SQL과 호환됨<br/>
기존 하이브 쿼리문을 스파크 SQL로 변환해야 하는 상황을 만날 수도 있음<br/>
다행히 대부분의 하이브 쿼리문을 스파크 SQL에서 바로 사용할 수 있음<br/>
다음은 **외부 테이블**을 생성하는 예제임<br/>
스파크는 외부 테이블의 메타데이터를 관리함<br/>
하지만 데이터 파일은 스파크에서 관리하지 않음<br/>
CREATE EXTERNAL TABLE 구문을 사용해 외부 테이블을 생성할 수 있음<br/>

다음 구문을 사용해 예제 데이터에 미리 저장된 데이터 파일의 내용을 테이블 형태로 볼 수 있음<br/>

In [131]:
/*
import java.io.File
import org.apache.spark.sql.{Row, SaveMode, SparkSession}

val warehouseLocation = new File("spark-warehouse").getAbsolutePath

val spark = SparkSession
  .builder()
  .appName("Spark Hive Example")
  .config("spark.sql.warehouse.dir", warehouseLocation)
  .enableHiveSupport()
  .getOrCreate()

import spark.implicits._
import spark.sql
*/

spark.sql(
"""
CREATE EXTERNAL TABLE IF NOT EXISTS
    hive_flights(DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count LONG)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
LOCATION 'Downloads/Spark-The-Definitive-Guide/data/flight-data-hive'
""")

org.apache.spark.sql.AnalysisException:  Hive support is required to CREATE Hive TABLE (AS SELECT);

*위 예제 코드가 에러가 나서 주석 코드를 통해 해결해보려 했으나 주석 코드를 포함시켜 실행해도 에러가 발생*

## 10.6.4 테이블에 데이터 삽입하기

In [132]:
spark.sql(
"""
SELECT count(DEST_COUNTRY_NAME) FROM flights_from_select
""").show()

spark.sql(
"""
INSERT INTO flights_from_select
    SELECT * FROM flights LIMIT 20
""")

spark.sql(
"""
SELECT count(DEST_COUNTRY_NAME) FROM flights_from_select
""").show()

+------------------------+
|count(DEST_COUNTRY_NAME)|
+------------------------+
|                       0|
+------------------------+

+------------------------+
|count(DEST_COUNTRY_NAME)|
+------------------------+
|                       0|
+------------------------+



*spark-warehouse 폴더 안에 flights_from_select 디렉터리는 잘 생성됐는데 위 코드는 왜 count해서 0이라는 이상한 결과를 뱉는 것인지?*

## 10.6.5 테이블 메타데이터 확인하기

In [133]:
spark.sql( // 메타 데이터 확인
"""
DESCRIBE TABLE flights
""").show()

+-------------------+---------+-------+
|           col_name|data_type|comment|
+-------------------+---------+-------+
|  DEST_COUNTRY_NAME|   string|   null|
|ORIGIN_COUNTRY_NAME|   string|   null|
|              count|   bigint|   null|
+-------------------+---------+-------+



In [134]:
spark.sql( // 메타 데이터 확인
"""
DESCRIBE TABLE flights_from_select
""").show()

+-------------------+---------+-------+
|           col_name|data_type|comment|
+-------------------+---------+-------+
|  DEST_COUNTRY_NAME|   string|   null|
|ORIGIN_COUNTRY_NAME|   string|   null|
|              count|   bigint|   null|
+-------------------+---------+-------+



### 기타 명령어
* DESCRIBE: 메타정보 확인
* SHOW PARTITIONS: 파티셔닝 스키마 정보 확인
* REFRESH: 캐싱된 항목을 갱신
* MSCK REPAIR TABLE: 관리하는 테이블의 파티션 정보를 새로 고침
* DROP TABLE: 외부 테이블을 제거하면 데이터는 삭제되지 않지만, 외부 테이블명을 이용해 데이터 조회할 수 없게 됨
* CACHE/UNCACHE TABLE

# 10.7 뷰

지금까지 테이블을 생성하는 방법을 알아봤고 이제 뷰를 정의해보겠음<br/>
뷰는 기존 테이블에 여러 트랜스포메이션 작업을 지정함<br/>
*기본적으로 뷰는 단순 쿼리 실행 계획일 뿐임*<br/>
뷰를 사용하면 로직을 체계화하거나 재사용하기 편하게 만들 수 있음<br/>
스파크는 뷰와 관련된 다양한 개념을 가지고 있음<br/>
뷰는 데이터베이스에 설정하는 전역 뷰나 세션별 뷰가 될 수 있음<br/>

## 10.7.1 뷰 생성하기

최종 사용자에게 뷰는 테이블처럼 보임<br/>
신규 경로에 모든 데이터를 다시 저장하는 대신 단순하게 쿼리 시점에 데이터 소스에 트랜스포메이션을 수행함<br/>
filter, select 또는 대규모 GROUP BY, ROLLUP 같은 트랜스포메이션이 이에 해당함<br/>
다음은 목적지가 United States인 항공운항 데이터를 보기 위한 뷰를 생성하는 예제임<br/>

In [135]:
spark.sql("""
    CREATE VIEW just_usa_view AS
        SELECT * FROM flights WHERE DEST_COUNTRY_NAME = 'United States'
        """).show()

++
||
++
++



테이블처럼 데이터베이스에 등록되지 않고 현재 세션에서만 사용할 수 있는 임시 뷰를 만들 수 있음

In [136]:
spark.sql("""
    CREATE TEMP VIEW just_usa_view_temp AS
        SELECT * FROM flights WHERE DEST_COUNTRY_NAME = 'United States'
        """).show()

org.apache.spark.sql.catalyst.analysis.TempTableAlreadyExistsException:  Temporary view 'just_usa_view_temp' already exists

전역적 임시 뷰(global temp view)도 만들 수 있음<br/>
전역적 임시 뷰는 데이터베이스에 상관 없이 사용할 수 있으므로 전체 스파크 애플리케이션에서 볼 수 있음<br/>
하지만 세션이 종료되면 뷰도 사라짐<br/>

In [137]:
spark.sql("""
    CREATE GLOBAL TEMP VIEW just_usa_global_view_temp AS
        SELECT * FROM flights WHERE DEST_COUNTRY_NAME = 'United States'
        """).show()

org.apache.spark.sql.catalyst.analysis.TempTableAlreadyExistsException:  Temporary view 'just_usa_global_view_temp' already exists

In [138]:
spark.sql("SHOW TABLES").show()

+--------+-------------------+-----------+
|database|          tableName|isTemporary|
+--------+-------------------+-----------+
| default|            flights|      false|
| default|flights_from_select|      false|
| default|      just_usa_view|      false|
| default|        nested_data|      false|
|        | just_usa_view_temp|       true|
|        |      some_sql_view|       true|
+--------+-------------------+-----------+



다음 예제에 나오는 키워드를 사용해 생성된 뷰를 덮어쓸 수 있음<br/>
임시 뷰와 일반 뷰 모두 덮어쓸 수 있음<br/>

In [139]:
spark.sql("""
    CREATE OR REPLACE TEMP VIEW just_usa_view_temp AS
        SELECT * FROM flights WHERE DEST_COUNTRY_NAME = 'United States'
        """).show()

++
||
++
++



이제 다른 테이블과 동일한 방식으로 뷰를 사용할 수 있음

In [140]:
spark.sql("SELECT * FROM just_usa_view_temp")

res135: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


뷰는 실질적으로 트랜스포메이션이며 스파크는 쿼리가 실행될 때 뷰에 정의된 트랜스포메이션을 수행함<br/>
즉, 테이블의 데이터를 실제로 조회하는 경우에만 필터를 적용함<br/>
사실 뷰는 기존 DataFrame에서 새로운 DataFrame을 만드는 것과 동일함<br/>

스파크 DataFrame과 스파크 SQL로 생성된 쿼리 실행 계획을 비교해 확인할 수 있음<br/>
DataFrame에서는 다음 코드를 사용함<br/>

In [141]:
val flights = spark.read.format("json")
    .load("Downloads/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json")
val just_usa_df = flights.where("dest_country_name = 'United States'")
just_usa_df.selectExpr("*").explain

== Physical Plan ==
*(1) Filter (isnotnull(dest_country_name#1243) AND (dest_country_name#1243 = United States))
+- FileScan json [DEST_COUNTRY_NAME#1243,ORIGIN_COUNTRY_NAME#1244,count#1245L] Batched: false, DataFilters: [isnotnull(DEST_COUNTRY_NAME#1243), (DEST_COUNTRY_NAME#1243 = United States)], Format: JSON, Location: InMemoryFileIndex[file:/home/hj/Downloads/Spark-The-Definitive-Guide/data/flight-data/json/2015-s..., PartitionFilters: [], PushedFilters: [IsNotNull(DEST_COUNTRY_NAME), EqualTo(DEST_COUNTRY_NAME,United States)], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:bigint>




flights: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]
just_usa_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


SQL 사용 시 다음과 같은 쿼리를 실행함

In [142]:
spark.sql("EXPLAIN SELECT * FROM just_usa_view").show(false)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|plan                                                                                                                                                                                                                                                                                                                                                                                     

다음 쿼리를 사용할 수도 있음

In [143]:
spark.sql("EXPLAIN SELECT * FROM flights WHERE dest_country_name = 'United States'").show(false)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|plan                                                                                                                                                                                                                                                                                                                                                                                     

따라서 DataFrame이나 SQL 중 가장 편한 방법을 선택해서 사용하면 됨

## 10.7.2 뷰 제거하기

테이블을 제거하는 것과 동일한 방식으로 뷰를 제거할 수 있음<br/>
뷰와 테이블 제거의 핵심 차이점은 뷰는 어떤 데이터도 제거되지 않으며 뷰 정의만 제거된다는 것임<br/>

In [144]:
spark.sql("DROP VIEW IF EXISTS just_usa_view")

res139: org.apache.spark.sql.DataFrame = []


# 10.8 데이터베이스

데이터베이스는 여러 테이블을 조직화하기 위한 도구임<br/>
앞서 언급했듯이 데이터베이스를 정의하지 않으면 스파크는 기본 데이터베이스를 사용함<br/>
스파크에서 실행하는 모든 SQL 명령문은 사용 중인 데이터베이스 범위에서 실행됨<br/>
즉, 데이터베이스를 변경하면 이전에 생성한 모든 사용자 테이블이 변경하기 전의 데이터베이스에 속해 있으므로 기존 테이블 데이터를 조회하려면 다르게 쿼리해야 함<br/>

*CAUTION*<br/>
특히 동료와 동일한 컨텍스트나 세션을 공유하는 경우 혼란을 일으킬 수 있으므로 사용할 데이터베이스를 반드시 설정해야 함<br/>

다음 명령을 사용해 전체 데이터베이스 목록을 확인할 수 있음<br/>

In [145]:
spark.sql("SHOW DATABASES").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



## 10.8.1 데이터베이스 생성하기

데이터베이스를 생성하는 방법은 다른 예제에서 사용한 방식과 동일한 패턴을 따름<br/>
다만 CREATE DATABASE 키워드를 사용해야 함<br/>

In [146]:
spark.sql("CREATE DATABASE some_db")

res141: org.apache.spark.sql.DataFrame = []


## 10.8.2 데이터베이스 설정하기

USE 키워드 다음에 데이터베이스명을 붙여서 쿼리 수행에 필요한 데이터베이스를 설정할 수 있음

In [147]:
spark.sql("USE some_db")

res142: org.apache.spark.sql.DataFrame = []


모든 쿼리는 테이블 이름을 찾을 때 앞서 지정한 데이터베이스를 참조함<br/>
하지만 다른 데이터베이스를 지정했기 때문에 정상 동작하던 쿼리가 실패하거나 다른 결과를 반환할 수도 있음<br/>

In [148]:
spark.sql("SHOW tables").show(false)

+--------+------------------+-----------+
|database|tableName         |isTemporary|
+--------+------------------+-----------+
|        |just_usa_view_temp|true       |
|        |some_sql_view     |true       |
+--------+------------------+-----------+



In [149]:
spark.sql("SELECT * FROM flights").show(false) // 테이블이나 뷰를 찾을 수 없으므로 에러 발생

org.apache.spark.sql.AnalysisException:  Table or view not found: flights; line 1 pos 14;

그러나 올바른 접두사를 사용해 다른 데이터베이스의 테이블에 쿼리를 수행할 수 있음<br/>

In [150]:
spark.sql("SELECT * FROM default.flights")

res145: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


다음 명령을 사용해 현재 어떤 데이터베이스를 사용 중인지 확인할 수 있음<br/>

In [151]:
spark.sql("SELECT current_database()").show

+------------------+
|current_database()|
+------------------+
|           some_db|
+------------------+



다음 명령을 사용해 기본 데이터베이스로 돌아갈 수 있음<br/>

In [152]:
spark.sql("USE default")

res147: org.apache.spark.sql.DataFrame = []


## 10.8.3 데이터베이스 제거하기

DROP DATABASE 키워드

In [153]:
spark.sql("DROP DATABASE IF EXISTS some_db")

res148: org.apache.spark.sql.DataFrame = []


# 10.10 고급 주제

지금까지 데이터가 어디에 존재하는지, 어떻게 구성해야 하는지 알아보았음<br/>
이제 데이터를 쿼리하는 방법을 알아보겠음<br/>
SQL 쿼리는 특정 명령 집합을 실행하도록 요청하는 SQL 구문임<br/>
SQL 구문은 조작, 정의, 제어와 관련된 명령을 정의할 수 있음<br/>
이 책에서는 대부분 조작과 관련된 내용을 다룸<br/>

## 10.10.1 복합 데이터 타입

복합 데이터 타입은 표준 SQL과는 거리가 있으며 표준 SQL에는 존재하지 않는 강력한 기능임<br/>
이를 SQL에서 어떻게 적절하게 처리하는지 이해할 필요가 있음<br/>
스파크 SQL에는 구조체, 리스트, 맵 3가지 핵심 복합 데이터 타입이 존재함<br/>

### 구조체

구조체는 맵에 더 가까우며 스파크에서 중첩 데이터를 생성하거나 쿼리하는 방법을 제공함<br/>
구조체를 만들기 위해서는 여러 컬럼이나 표현식을 괄호로 묶기만 하면 됨<br/>

In [154]:
spark.sql("""
    CREATE VIEW IF NOT EXISTS nested_data AS
        SELECT (DEST_COUNTRY_NAME, ORIGIN_COUNTRY_NAME) as country, count FROM flights""")

res149: org.apache.spark.sql.DataFrame = []


이제 다음과 같이 구조체 데이터를 조회할 수 있음<br/>

In [155]:
spark.sql("SELECT * FROM nested_data")

res150: org.apache.spark.sql.DataFrame = [country: struct<DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string>, count: bigint]


구조체의 개별 컬럼을 조회할 수도 있음<br/>

In [156]:
spark.sql("SELECT country.DEST_COUNTRY_NAME, count FROM nested_data")

res151: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, count: bigint]


구조체의 이름과 모든 하위 컬럼을 지정해 모든 값을 조회할 수 있음<br/>
진짜 하위 컬럼은 아니지만 모든 것을 마치 컬럼처럼 다룰 수 있기 때문에 다음과 같이 더 간단한 방법을 사용할 수도 있음<br/>

In [157]:
spark.sql("SELECT country.*, count FROM nested_data")

res152: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


### 리스트

값의 리스트를 만드는 collect_list 함수나 중복 값이 없는 배열을 만드는 collect_set 함수를 사용할 수 있음<br/>
두 함수 모두 집계 함수이므로 집계 연산 시에만 사용할 수 있음<br/>

In [158]:
spark.sql("""
SELECT DEST_COUNTRY_NAME as new_name, collect_list(count) as flight_counts,
    collect_set(ORIGIN_COUNTRY_NAME) as origin_set
FROM flights GROUP BY DEST_COUNTRY_NAME
""")

res153: org.apache.spark.sql.DataFrame = [new_name: string, flight_counts: array<bigint> ... 1 more field]


## 10.10.2 함수

In [160]:
spark.sql("SHOW FUNCTIONS").show(false)

+--------+
|function|
+--------+
|!       |
|!=      |
|%       |
|&       |
|*       |
|+       |
|-       |
|/       |
|<       |
|<=      |
|<=>     |
|<>      |
|=       |
|==      |
|>       |
|>=      |
|^       |
|abs     |
|acos    |
|acosh   |
+--------+
only showing top 20 rows



In [161]:
spark.sql("SHOW SYSTEM FUNCTIONS").show(false)

+--------+
|function|
+--------+
|!       |
|!=      |
|%       |
|&       |
|*       |
|+       |
|-       |
|/       |
|<       |
|<=      |
|<=>     |
|<>      |
|=       |
|==      |
|>       |
|>=      |
|^       |
|abs     |
|acos    |
|acosh   |
+--------+
only showing top 20 rows



In [162]:
spark.sql("SHOW USER FUNCTIONS").show(false)

+--------+
|function|
+--------+
+--------+



In [164]:
spark.sql("SHOW FUNCTIONS 's*'").show(false)

+------------------+
|function          |
+------------------+
|schema_of_csv     |
|schema_of_json    |
|second            |
|sentences         |
|sequence          |
|sha               |
|sha1              |
|sha2              |
|shiftleft         |
|shiftright        |
|shiftrightunsigned|
|shuffle           |
|sign              |
|signum            |
|sin               |
|sinh              |
|size              |
|skewness          |
|slice             |
|smallint          |
+------------------+
only showing top 20 rows



In [165]:
spark.sql("SHOW FUNCTIONS LIKE 'collect*'").show(false)

+------------+
|function    |
+------------+
|collect_list|
|collect_set |
+------------+



### 사용자 정의 함수

## 10.10.3 서브쿼리

### 비상호연관 조건절 서브쿼리

### 상호연관 조건절 서브쿼리

### 비상호연관 스칼라 쿼리

# 10.11 다양한 기능

## 10.11.1 설정

## 10.11.2 SQL에서 설정값 지정하기

# 10.12 정리

스파크 SQL과 DataFrame은 매우 밀접하게 연관되어 있음<br/>
이 장에서는 스파크 SQL과 관련된 세부사항을 자세히 알아보았음<br/>
다음 장에서는 Dataset에 대해 집중적으로 알아보겠음<br/>